In [5]:
from cupy import ndarray
from typing import List, Any, Tuple, Set

import cupy
import nodeleys as ndl
import sys
import copy
import tensorflow as tf

def t_update(ndl):
  pending_for_deletion = []
  for i in sys.modules:
    if i.split('.')[0] == 'nodeleys':
      pending_for_deletion.append(i)

  # print(pending_for_deletion)
    
  for i in pending_for_deletion:
    del sys.modules[i]

  import nodeleys as ndl
  return ndl

In [6]:
import time

ndl = t_update(ndl)
from nodeleys.system.system_func import *
from nodeleys.graph import Switch

data_x = ndl.Node(cupy.random.normal(size=(1516,1516)), name='x')
data_y = ndl.Node(cupy.random.normal(size=(1516,1)), name='y')
data_y.is_constant = True

w0 = ndl.Node(cupy.random.normal(size=(1516,516)), name='w0')
b0 = ndl.Node(cupy.random.normal(size=(1,516)), name='b0')

w1 = ndl.Node(cupy.random.normal(size=(516,64)), name='w1')
b1 = ndl.Node(cupy.random.normal(size=(1,64)), name='b1')

w1r = ndl.Node(cupy.random.normal(size=(516,64)), name='w1r')
b1r = ndl.Node(cupy.random.normal(size=(1,64)), name='b1r')

rR = ndl.Node(cupy.random.normal(size=(64,64)), name='rR')
bR = ndl.Node(cupy.random.normal(size=(1,64)), name='bR')

rL = ndl.Node(cupy.random.normal(size=(64,64)), name='rL')
bL = ndl.Node(cupy.random.normal(size=(1,64)), name='bL')

w2 = ndl.Node(cupy.random.normal(size=(64,1)), name='w2')
b2 = ndl.Node(cupy.random.normal(size=(1,1)), name='b2')

RANX = ndl.Node(cupy.random.normal(size=(1516, 64)), name='RANX')

gamma0 = node_matmul(data_x, w0, 'gamma0')
y0 = node_add(b0, gamma0, 'y0')

gamma1 = node_matmul(y0, w1, 'gamma1')
y1 = node_add(gamma1, b1, 'y1')

gamma1r = node_matmul(y0, w1r, 'gamma1r')
y1rs = node_add(gamma1r, b1r, 'y1rs')

y1r = Switch(y1rs, [gamma1r], [node_mul(y1rs, 0.), node_mul(RANX, y1rs)], ['x<=0', 'x>0'], 'y1r').compile()
# y1r = Virtual([y1r_pre], [node_mul(y1r_pre, 0.), y1r_pre], ['[0]<0', '[0]>=0'], 'y1r').compile()

gamma_rR = node_matmul(y1r, rR, 'gamma_rR')
y_rR = node_add(gamma_rR, bR, 'y_rR')

gamma_rL = node_matmul(y1r, rL, 'gamma_rL')
y_rL = node_add(gamma_rL, bL, 'y_rL')

y_rRL = node_add(y_rR, y_rL, 'y_rRL')

merge = node_add(y1r, y_rRL, 'merge')
merge2 = node_add(merge, y1, 'merge2')

gamma2 = node_matmul(merge2, w2, 'gamma2')
y2 = node_add(gamma2, b2, 'y2')

diff = node_sub(y2, data_y, 'diff')
squared = node_pow(diff, 2., 'squared')
redsum = node_redsum(squared, 0, 'redsum')
loss = node_div(redsum, 16., 'loss')

with tf.GradientTape(persistent=True) as g:
  tf_data_x = tf.Variable(tf.convert_to_tensor(data_x.tensor.get()))
  tf_data_y = tf.Variable(tf.convert_to_tensor(data_y.tensor.get()))
  tf_w0 = tf.Variable(tf.convert_to_tensor(w0.tensor.get()))
  tf_b0 = tf.Variable(tf.convert_to_tensor(b0.tensor.get()))
  tf_w1 = tf.Variable(tf.convert_to_tensor(w1.tensor.get()))
  tf_b1 = tf.Variable(tf.convert_to_tensor(b1.tensor.get()))
  tf_w1r = tf.Variable(tf.convert_to_tensor(w1r.tensor.get()))
  tf_b1r = tf.Variable(tf.convert_to_tensor(b1r.tensor.get()))
  tf_rR = tf.Variable(tf.convert_to_tensor(rR.tensor.get()))
  tf_bR = tf.Variable(tf.convert_to_tensor(bR.tensor.get()))
  tf_rL = tf.Variable(tf.convert_to_tensor(rL.tensor.get()))
  tf_bL = tf.Variable(tf.convert_to_tensor(bL.tensor.get()))
  tf_w2 = tf.Variable(tf.convert_to_tensor(w2.tensor.get()))
  tf_b2 = tf.Variable(tf.convert_to_tensor(b2.tensor.get()))
  tf_RANX = tf.Variable(tf.convert_to_tensor(RANX.tensor.get()))

  tf_gamma0 = tf_data_x @ tf_w0
  tf_y0 = tf_gamma0 + tf_b0
  tf_gamma1 = tf_y0 @ tf_w1
  tf_y1 = tf_gamma1 + tf_b1
  tf_gamma1r = tf_y0 @ tf_w1r
  tf_gamma1rs = tf_y0 @ tf_w1r
  print(id(tf_gamma1r))
  print(id(tf_gamma1rs))
  tf_y1rs = tf_gamma1r + tf_b1r
  tf_y1r = tf.where(tf_y1rs <= 0, x=0., y=tf_RANX * tf_y1rs)


  tf_gamma_rR = tf_y1r @ tf_rR
  tf_y_rR = tf_gamma_rR + tf_bR

  tf_gamma_rL = tf_y1r @ tf_rL
  tf_y_rL = tf_gamma_rL + tf_bL

  tf_y_rRL = tf_y_rR + tf_y_rL

  tf_merge = tf_y1r + tf_y_rRL
  tf_merge2 = tf_merge + tf_y1
  tf_gamma2 = tf_merge2 @ tf_w2
  tf_y2 = tf_gamma2 + tf_b2
  tf_diff = tf_y2 - tf_data_y
  tf_squared = tf_diff ** 2
  tf_redsum = tf.reduce_sum(tf_squared, axis=0, keepdims=True)
  tf_loss = tf_redsum / 16.
  
start_time = time.time()
grads = g.gradient(tf_loss, [tf_w0, tf_w1, tf_w2, tf_b0, tf_gamma_rR])
print("--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
loss.adic.set_as_objective()
loss.adic.begin_backprop()
w0.get_gradient()
w1.get_gradient()
w2.get_gradient()
b0.get_gradient()
print("--- %s seconds ---" % (time.time() - start_time))

140028226853136
140028226853520
--- 0.0042726993560791016 seconds ---
--- 0.012156486511230469 seconds ---


In [7]:
w0.get_gradient()

array([[ 12390472.87312919,   2139131.91203419, -21229394.73145704, ...,
          -973567.96846739,   2584570.5112279 ,   2912612.62076973],
       [   730617.60103639,   1535289.5466834 ,  -5044188.1051876 , ...,
          -702132.47946235,   8383366.10131261,    963839.97658494],
       [  2984902.28988521,  16281350.42240268,  20232138.70030956, ...,
         -5973123.29090067,  -8639237.57250413, -31459096.37217195],
       ...,
       [  4772746.07457647, -11624671.07809455,  -7092918.97663552, ...,
         -1188942.02413083,  15188460.85241094,  -3854844.37347624],
       [ -5415575.0306724 ,  -8067215.95782039, -24026489.18829124, ...,
         -3933778.937632  ,  21567098.63651668,  41075402.93925983],
       [ -9177572.96674252,  13799767.03081999,  11840310.77443176, ...,
         17526664.18345317,    922063.77566141, -12374457.88267857]])

In [8]:
grads[0]

<tf.Tensor: shape=(1516, 516), dtype=float64, numpy=
array([[ 12390472.87312918,   2139131.91203419, -21229394.73145703, ...,
          -973567.9684674 ,   2584570.51122788,   2912612.62076971],
       [   730617.60103637,   1535289.54668338,  -5044188.10518761, ...,
          -702132.47946235,   8383366.10131256,    963839.97658497],
       [  2984902.28988521,  16281350.4224027 ,  20232138.70030963, ...,
         -5973123.29090064,  -8639237.57250413, -31459096.37217199],
       ...,
       [  4772746.07457647, -11624671.07809454,  -7092918.97663551, ...,
         -1188942.02413083,  15188460.85241092,  -3854844.37347623],
       [ -5415575.03067242,  -8067215.95782037, -24026489.1882912 , ...,
         -3933778.93763197,  21567098.63651669,  41075402.93925986],
       [ -9177572.96674253,  13799767.03081998,  11840310.77443176, ...,
         17526664.18345312,    922063.7756614 , -12374457.88267863]])>

In [26]:
y1.tensor

array([[ -592.80803381,  1758.0489002 ,    38.55142327, ...,
         -240.78185257,  -927.29209325,  -177.74273524],
       [ 1121.06165473,   225.50427553,  -795.03607837, ...,
          -56.40766872,  -713.57003717,  -115.44455186],
       [  496.85798588,   462.03444507,  -169.46510907, ...,
         1429.10801311,  -978.65279944,  -965.4081674 ],
       ...,
       [ 1818.18485501,   377.66176783,  -928.11346932, ...,
         1872.43182606,    23.89248882,   187.20460361],
       [  172.07079311,  1239.20081593,   -42.36847309, ...,
         1169.62770012,   187.40491184,  1231.00144376],
       [ -693.07603054,   192.02241137,  -209.65612244, ...,
        -1076.73504476,  -459.67665045,  -213.56274296]])

In [35]:
softmax_RANX = tf.math.pow(2.7182, RANX.tensor.get()) / tf.math.reduce_sum(tf.math.pow(2.7182, RANX.tensor.get()), axis=1, keepdims=True)
softmax_RANX

<tf.Tensor: shape=(1516, 64), dtype=float32, numpy=
array([[0.02998197, 0.01165974, 0.00654347, ..., 0.02536652, 0.03159917,
        0.00160397],
       [0.00842244, 0.00736548, 0.02460846, ..., 0.01018342, 0.01251902,
        0.00819403],
       [0.0368217 , 0.02055938, 0.00604289, ..., 0.01702051, 0.00322675,
        0.00610522],
       ...,
       [0.01166366, 0.00245767, 0.02337791, ..., 0.01227763, 0.03284488,
        0.00408779],
       [0.00724606, 0.00752893, 0.05104459, ..., 0.00420289, 0.00103354,
        0.00346272],
       [0.03427997, 0.0105774 , 0.00888279, ..., 0.00255882, 0.03711211,
        0.00567789]], dtype=float32)>

In [36]:
tf.reduce_sum(softmax_RANX, axis=1)

<tf.Tensor: shape=(1516,), dtype=float32, numpy=
array([1.        , 1.        , 1.        , ..., 1.        , 1.        ,
       0.99999994], dtype=float32)>

In [37]:
softmax = node_div(node_pow(2.7182, RANX), node_redsum(node_pow(2.7182, RANX), 1))

In [42]:
x = node_add(1., softmax)

In [45]:
x.adic.prev

(None, Triplet(, , ; /))

In [40]:
node_redsum(softmax, 1).tensor

array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]])